In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score # 정확도 함수
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


In [3]:
Grandmaster = pd.read_csv('Dataset/perMinuteDataset/10min/Grandmaster.csv')

In [4]:
win_Grandmaster = Grandmaster[['Diff_FirstBLOOD', 'Diff_FirstDRAGON',
       'Diff_FirstHERALD', 'Diff_Firsttower', 'dragonType', 'WIN_invadeKill', 'WIN_invadeDeath','WIN_controlWARDPlaced', 
       'WIN_Kill_top', 'WIN_Kill_jgl', 'WIN_Kill_mid', 'WIN_Kill_ad', 'WIN_Kill_sup',
       'WIN_Death_top', 'WIN_Death_jgl', 'WIN_Death_mid', 'WIN_Death_ad', 'WIN_Death_sup', 
       'WIN_Asisst_top', 'WIN_Asisst_jgl', 'WIN_Asisst_mid', 'WIN_Asisst_ad', 'WIN_Asisst_sup',
       'WIN_LV_top', 'WIN_LV_jgl', 'WIN_LV_mid', 'WIN_LV_ad', 'WIN_LV_sup',
       'WIN_CS_top', 'WIN_CS_jgl', 'WIN_CS_mid', 'WIN_CS_ad', 'WIN_CS_sup',
       'WIN_jglCS_top', 'WIN_jglCS_jgl', 'WIN_jglCS_mid', 'WIN_jglCS_ad', 'WIN_jglCS_sup',
       'WIN_GOLD_top', 'WIN_GOLD_jgl', 'WIN_GOLD_mid', 'WIN_GOLD_ad', 'WIN_GOLD_sup',
       'WIN_WARDkill', 'WIN_Inhibitor','WIN_TOWERkill', 'WIN_WARDplaced']]
lose_Grandmaster = Grandmaster[['Diff_FirstBLOOD', 'Diff_FirstDRAGON',
       'Diff_FirstHERALD', 'Diff_Firsttower', 'dragonType',
       'LOSE_invadeDeath', 'LOSE_invadeKill',
       'LOSE_controlWARDPlaced',
       'LOSE_Kill_top', 'LOSE_Kill_jgl', 'LOSE_Kill_mid', 'LOSE_Kill_ad', 'LOSE_Kill_sup',
       'LOSE_Death_top', 'LOSE_Death_jgl',
       'LOSE_Death_mid', 'LOSE_Death_ad', 'LOSE_Death_sup',
       'LOSE_Asisst_top', 'LOSE_Asisst_jgl', 'LOSE_Asisst_mid',
       'LOSE_Asisst_ad', 'LOSE_Asisst_sup',
       'LOSE_LV_top', 'LOSE_LV_jgl',
       'LOSE_LV_mid', 'LOSE_LV_ad', 'LOSE_LV_sup',
       'LOSE_CS_top', 'LOSE_CS_jgl',
       'LOSE_CS_mid', 'LOSE_CS_ad', 'LOSE_CS_sup',
       'LOSE_jglCS_top', 'LOSE_jglCS_jgl', 'LOSE_jglCS_mid', 'LOSE_jglCS_ad', 'LOSE_jglCS_sup',
       'LOSE_GOLD_top', 'LOSE_GOLD_jgl',
       'LOSE_GOLD_mid', 'LOSE_GOLD_ad', 'LOSE_GOLD_sup',
       'LOSE_WARDkill', 'LOSE_Inhibitor',
       'LOSE_TOWERkill', 'LOSE_WARDplaced']]
colName = 'WIN'
win_Grandmaster = win_Grandmaster.rename(columns={f'{colName}_invadeKill': 'invadeKill', f'{colName}_invadeDeath': 'invadeDeath', 
                                                  f'{colName}_controlWARDPlaced': 'controlWARDPlaced',
                                                  f'{colName}_Kill_top': 'Kill_top',f'{colName}_Kill_jgl': 'Kill_jgl',f'{colName}_Kill_mid': 'Kill_mid',f'{colName}_Kill_ad': 'Kill_ad', f'{colName}_Kill_sup': 'Kill_sup',
                                                  f'{colName}_Death_top': 'Death_top',f'{colName}_Death_jgl': 'Death_jgl',f'{colName}_Death_mid': 'Death_mid',f'{colName}_Death_ad': 'Death_ad',f'{colName}_Death_sup': 'Death_sup',
                                                  f'{colName}_Asisst_top': 'Assist_top',f'{colName}_Asisst_jgl': 'Assist_jgl',f'{colName}_Asisst_mid': 'Assist_mid',f'{colName}_Asisst_ad': 'Assist_ad',f'{colName}_Asisst_sup': 'Assist_sup',
                                                  f'{colName}_LV_top': 'LV_top',f'{colName}_LV_jgl': 'LV_jgl',f'{colName}_LV_mid': 'LV_mid',f'{colName}_LV_ad': 'LV_ad',f'{colName}_LV_sup': 'LV_sup',
                                                  f'{colName}_CS_top': 'CS_top',f'{colName}_CS_jgl': 'CS_jgl',f'{colName}_CS_mid': 'CS_mid',f'{colName}_CS_ad': 'CS_ad',f'{colName}_CS_sup': 'CS_sup',
                                                  f'{colName}_jglCS_top': 'jglCS_top',f'{colName}_jglCS_jgl': 'jglCS_jgl',f'{colName}_jglCS_mid': 'jglCS_mid',f'{colName}_jglCS_ad': 'jglCS_ad',f'{colName}_jglCS_sup': 'jglCS_sup',
                                                  f'{colName}_GOLD_top': 'GOLD_top',f'{colName}_GOLD_jgl': 'GOLD_jgl',f'{colName}_GOLD_mid': 'GOLD_mid',f'{colName}_GOLD_ad': 'GOLD_ad',f'{colName}_GOLD_sup': 'GOLD_sup',
                                                  f'{colName}_WARDkill': 'WARDkill',f'{colName}_Inhibitor': 'Inhibitor',f'{colName}_TOWERkill': 'TOWERkill',f'{colName}_WARDplaced': 'WARDplaced'})

colName = 'LOSE'
lose_Grandmaster = lose_Grandmaster.rename(columns={f'{colName}_invadeKill': 'invadeKill', f'{colName}_invadeDeath': 'invadeDeath', 
                                                  f'{colName}_controlWARDPlaced': 'controlWARDPlaced',
                                                  f'{colName}_Kill_top': 'Kill_top',f'{colName}_Kill_jgl': 'Kill_jgl',f'{colName}_Kill_mid': 'Kill_mid',f'{colName}_Kill_ad': 'Kill_ad', f'{colName}_Kill_sup': 'Kill_sup',
                                                  f'{colName}_Death_top': 'Death_top',f'{colName}_Death_jgl': 'Death_jgl',f'{colName}_Death_mid': 'Death_mid',f'{colName}_Death_ad': 'Death_ad',f'{colName}_Death_sup': 'Death_sup',
                                                  f'{colName}_Asisst_top': 'Assist_top',f'{colName}_Asisst_jgl': 'Assist_jgl',f'{colName}_Asisst_mid': 'Assist_mid',f'{colName}_Asisst_ad': 'Assist_ad',f'{colName}_Asisst_sup': 'Assist_sup',
                                                  f'{colName}_LV_top': 'LV_top',f'{colName}_LV_jgl': 'LV_jgl',f'{colName}_LV_mid': 'LV_mid',f'{colName}_LV_ad': 'LV_ad',f'{colName}_LV_sup': 'LV_sup',
                                                  f'{colName}_CS_top': 'CS_top',f'{colName}_CS_jgl': 'CS_jgl',f'{colName}_CS_mid': 'CS_mid',f'{colName}_CS_ad': 'CS_ad',f'{colName}_CS_sup': 'CS_sup',
                                                  f'{colName}_jglCS_top': 'jglCS_top',f'{colName}_jglCS_jgl': 'jglCS_jgl',f'{colName}_jglCS_mid': 'jglCS_mid',f'{colName}_jglCS_ad': 'jglCS_ad',f'{colName}_jglCS_sup': 'jglCS_sup',
                                                  f'{colName}_GOLD_top': 'GOLD_top',f'{colName}_GOLD_jgl': 'GOLD_jgl',f'{colName}_GOLD_mid': 'GOLD_mid',f'{colName}_GOLD_ad': 'GOLD_ad',f'{colName}_GOLD_sup': 'GOLD_sup',
                                                  f'{colName}_WARDkill': 'WARDkill',f'{colName}_Inhibitor': 'Inhibitor',f'{colName}_TOWERkill': 'TOWERkill',f'{colName}_WARDplaced': 'WARDplaced'})


In [6]:
win_Grandmaster['result'] = 1
lose_Grandmaster['result'] = -1
display(win_Grandmaster)
display(lose_Grandmaster)

,Diff_FirstBLOOD,Diff_FirstDRAGON,Diff_FirstHERALD,Diff_Firsttower,dragonType,invadeKill,invadeDeath,controlWARDPlaced,Kill_top,Kill_jgl,...,GOLD_top,GOLD_jgl,GOLD_mid,GOLD_ad,GOLD_sup,WARDkill,Inhibitor,TOWERkill,WARDplaced,result
0,1,1,0,0,1,0,0,2,1,0,...,3140,3847,4667,3146,2153,1,0,0,16,1
1,1,0,1,1,0,4,2,6,2,3,...,3984,4954,3307,7224,3760,8,0,1,23,1
2,1,0,1,0,0,0,0,6,1,0,...,3519,3426,3685,4474,3328,5,0,0,18,1
3,1,0,1,0,0,1,0,5,0,5,...,3324,4724,3569,3797,2909,5,0,0,43,1
4,1,-1,1,0,5,0,0,3,2,0,...,3966,3242,3444,3759,2476,3,0,0,16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6262,-1,-1,1,0,6,0,0,4,1,0,...,3768,3319,3368,2713,2748,5,0,0,14,1
6263,-1,0,0,0,0,1,2,5,0,3,...,3422,4139,3615,3661,2525,5,0,0,20,1
6264,1,-1,0,0,2,0,0,6,2,2,...,3468,3952,3389,3917,2959,4,0,0,19,1
6265,-1,-1,-1,0,2,0,0,3,1,2,...,3883,3789,3619,3753,2152,2,0,0,14,1


,Diff_FirstBLOOD,Diff_FirstDRAGON,Diff_FirstHERALD,Diff_Firsttower,dragonType,invadeDeath,invadeKill,controlWARDPlaced,Kill_top,Kill_jgl,...,GOLD_top,GOLD_jgl,GOLD_mid,GOLD_ad,GOLD_sup,WARDkill,Inhibitor,TOWERkill,WARDplaced,result
0,1,1,0,0,1,0,0,0,0,2,...,3136,4004,3577,2998,2212,1,0,0,11,-1
1,1,0,1,1,0,4,2,2,0,1,...,2893,3262,3453,3317,2224,2,0,0,17,-1
2,1,0,1,0,0,0,0,5,0,0,...,3011,3123,4046,3719,3059,4,0,0,34,-1
3,1,0,1,0,0,1,0,8,0,1,...,3336,3673,4176,3456,2749,11,0,0,21,-1
4,1,-1,1,0,5,0,0,5,2,2,...,3221,3970,3156,3451,2523,2,0,0,19,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6262,-1,-1,1,0,6,0,0,7,0,4,...,3026,3916,3238,5490,2941,6,0,0,30,-1
6263,-1,0,0,0,0,1,2,7,0,2,...,2991,3798,3168,4714,2520,6,0,0,28,-1
6264,1,-1,0,0,2,0,0,3,0,2,...,2914,3708,3557,3556,2735,3,0,0,20,-1
6265,-1,-1,-1,0,2,0,0,3,0,1,...,2884,3864,3932,4804,2437,3,0,0,19,-1


In [7]:
rf = RandomForestClassifier(n_estimators=500, max_depth=50, random_state = 10)
lgbm = LGBMClassifier(n_estimators=100, verbosity=0)
cat = CatBoostClassifier(iterations=2, depth=2, learning_rate=1)
et = ExtraTreesClassifier(n_estimators=100, random_state = 10)
data = pd.concat([win_Grandmaster, lose_Grandmaster], axis=0, ignore_index=True)
data


,Diff_FirstBLOOD,Diff_FirstDRAGON,Diff_FirstHERALD,Diff_Firsttower,dragonType,invadeKill,invadeDeath,controlWARDPlaced,Kill_top,Kill_jgl,...,GOLD_top,GOLD_jgl,GOLD_mid,GOLD_ad,GOLD_sup,WARDkill,Inhibitor,TOWERkill,WARDplaced,result
0,1,1,0,0,1,0,0,2,1,0,...,3140,3847,4667,3146,2153,1,0,0,16,1
1,1,0,1,1,0,4,2,6,2,3,...,3984,4954,3307,7224,3760,8,0,1,23,1
2,1,0,1,0,0,0,0,6,1,0,...,3519,3426,3685,4474,3328,5,0,0,18,1
3,1,0,1,0,0,1,0,5,0,5,...,3324,4724,3569,3797,2909,5,0,0,43,1
4,1,-1,1,0,5,0,0,3,2,0,...,3966,3242,3444,3759,2476,3,0,0,16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12529,-1,-1,1,0,6,0,0,7,0,4,...,3026,3916,3238,5490,2941,6,0,0,30,-1
12530,-1,0,0,0,0,2,1,7,0,2,...,2991,3798,3168,4714,2520,6,0,0,28,-1
12531,1,-1,0,0,2,0,0,3,0,2,...,2914,3708,3557,3556,2735,3,0,0,20,-1
12532,-1,-1,-1,0,2,0,0,3,0,1,...,2884,3864,3932,4804,2437,3,0,0,19,-1


In [10]:
colCnt = data.shape[1]
print(colCnt)
X = data.iloc[:, :colCnt-1]
y = data.iloc[:, colCnt-1:]
# # features/target, train/test dataset 분리
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할|
train_y = train_y.values.ravel()
rf.fit(train_x, train_y)
rf_pre = rf.predict(test_x)
tn, fp, fn, tp = confusion_matrix(test_y, rf_pre).ravel()
print("RandomForest :", round(accuracy_score(test_y, rf_pre)*100, 2), "%")
print("RandomForest f1 :", round(f1_score(test_y, rf_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)
lgbm.fit(train_x, train_y)
lgbm_pre = lgbm.predict(test_x)
tn, fp, fn, tp = confusion_matrix(test_y, lgbm_pre).ravel()
print("lightGBM :", round(accuracy_score(test_y, lgbm_pre)*100, 2), "%")
print("lightGBM f1 :", round(f1_score(test_y, lgbm_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)
cat.fit(train_x, train_y)
cat_pre = cat.predict(test_x)
tn, fp, fn, tp = confusion_matrix(test_y, cat_pre).ravel()
print("CatBoost :", round(accuracy_score(test_y, cat_pre)*100, 2), "%")
print("CatBoost f1 :", round(f1_score(test_y, cat_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)
et = ExtraTreesClassifier(n_estimators=100, random_state = 10)
et.fit(train_x, train_y)
et_pre = et.predict(test_x)
tn, fp, fn, tp = confusion_matrix(test_y, et_pre).ravel()
print("ExtraTrees :", round(accuracy_score(test_y, et_pre)*100, 2), "%")
print("ExtraTrees f1 :", round(f1_score(test_y, et_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)

RandomForest : 73.63 %
RandomForest f1 : 73.8 %
tn: 915  fp: 320  fn: 341  tp: 931
lightGBM : 78.98 %
lightGBM f1 : 79.03 %
tn: 987  fp: 248  fn: 279  tp: 993
0:	learn: 0.6595127	total: 137ms	remaining: 137ms
1:	learn: 0.6315960	total: 150ms	remaining: 0us
CatBoost : 61.63 %
CatBoost f1 : 55.95 %
tn: 934  fp: 301  fn: 661  tp: 611
ExtraTrees : 77.18 %
ExtraTrees f1 : 76.95 %
tn: 980  fp: 255  fn: 317  tp: 955
